In [1]:
import requests
from pymongo import MongoClient
import pandas as pd
import random

# Create user

In [2]:
personajes = ['Barack Obama', 'Voldemort', 'Chewbacca', 'Angela Merkel','Margherita Hack','Valentino Rossi', 
              'must4in3', 'Aretha Franklin', 'Marge Simpson', 'Darth Vader', 'Yoda', 'R2-D2', 'John Belushi',
              'Maccio Capatonda', ' Elon Musk', 'Che Guevara', 'Gollum', 'Samwise Gamgee', 'Jon Snow','Arya Stark',
              'Heidi', 'Hermione', 'Pikachu', 'Topolino', 'Berlusconi', 'Greta Thunberg','Hulk', 'Dilan Dog', 
              'Stephen Hopkins','Oprah Winfre', 'Frida Kahlo','Tyrion Lannister', 'Jennifer Aniston']

for personage in personajes:
    res = requests.get(f'http://localhost:3000/user/create/{personage}')

In [3]:
print(res)

<Response [500]>


In [4]:
res.json()

{'message': "name 'db' is not defined", 'status': 'error'}

In [5]:
# check users in Mongo
dbName = "api_chat_sentiment_analysis"
mongodbURL = f"mongodb://localhost/{dbName}"
client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()

In [6]:
cur = db.user.find({"username":{"$ne": None}})

In [7]:
df = pd.DataFrame(cur)
df.head()

,_id,username
0,5f0ca75139a4fd996c03d531,Barack Obama
1,5f0ca75139a4fd996c03d532,Voldemort
2,5f0ca75139a4fd996c03d533,Chewbacca
3,5f0ca75139a4fd996c03d534,Angela Merkel
4,5f0ca75139a4fd996c03d535,Margherita Hack


# Create Chat

In [8]:
df["_id"]= df["_id"].astype(str) 

In [9]:
five_chats = ['How I met your mother', 'Dumbledore’s Army', 
              'Tesla SpaceX','Italian restaurants in Madrid','datamad0620']
chats_room = dict()
for i,chat in enumerate(five_chats):   
    chats_room[f'{chat}']= {'user_ids': [random.choice([row for row in df['_id']]) for _ in range(5)]}

In [10]:
for room,user_ids in chats_room.items():
    res = requests.get(f'http://localhost:3000/chat/create/{room}?user_id={user_ids.values()}')

In [11]:
print(res)

<Response [500]>


In [12]:
res.json()

{'message': "Error. users_ids doesn't exist in database. It is not possible to create an empty chat",
 'status': 'api-error'}

In [13]:
cur = db.chatItem.find({"chat_name":{"$ne": None}})

In [14]:
df_chat = pd.DataFrame(cur)
df_chat.head()

,_id,chat_name,users_ids,messages
0,5f0ca76c39a4fd996c03d552,How I met your mother,"[5f0ca75239a4fd996c03d551, 5f0ca75239a4fd996c0...","[{'username': 'Jennifer Aniston', 'user_id': '..."
1,5f0ca76d39a4fd996c03d553,Dumbledore’s Army,"[5f0ca75139a4fd996c03d532, 5f0ca75239a4fd996c0...","[{'username': 'Voldemort', 'user_id': '5f0ca75..."
2,5f0ca76d39a4fd996c03d554,Tesla SpaceX,"[5f0ca75239a4fd996c03d542, 5f0ca75139a4fd996c0...","[{'username': 'Valentino Rossi', 'user_id': '5..."
3,5f0ca76d39a4fd996c03d555,Italian restaurants in Madrid,"[5f0ca75139a4fd996c03d534, 5f0ca75139a4fd996c0...","[{'username': 'Angela Merkel', 'user_id': '5f0..."
4,5f0ca76d39a4fd996c03d556,datamad0620,"[5f0ca75239a4fd996c03d542, 5f0ca75239a4fd996c0...","[{'username': 'must4in3', 'user_id': '5f0ca751..."


# Add user in a Chat

In [15]:
df_chat["_id"]= df_chat["_id"].astype(str) 

In [16]:
user_random = random.choice([row for row in df['_id']])
conversation_random = random.choice([row for row in df_chat['_id']])

In [17]:
res = requests.get(f'http://localhost:3000/chat/{conversation_random}/adduser?user_id={user_random}')

In [18]:
print(res)

<Response [500]>


In [19]:
res.json()

{'message': "Error. chat doesn't exist", 'status': 'api-error'}

# Add message in chat

kaggle dataset quote
https://www.kaggle.com/coolcoder22/quotes-dataset

In [20]:
quote = pd.read_csv('inputs/quote.csv', encoding = 'latin-1')

In [21]:
quote

,Auther,quote
0,A. A. Milne,"If you live to be a hundred, I want to live to..."
1,A. A. Milne,Promise me you'll always remember: You're brav...
2,A. A. Milne,"Did you ever stop to think, and forget to star..."
3,A. A. Milne,Organizing is what you do before you do someth...
4,A. A. Milne,"Weeds are flowers too, once you get to know them."
...,...,...
36160,Zsa Zsa Gabor,"I know nothing about sex, because I was always..."
36161,Zsa Zsa Gabor,Conrad Hilton was very generous to me in the d...
36162,Zsa Zsa Gabor,The women's movement hasn't changed my sex lif...
36163,Zsa Zsa Gabor,I always liked parties. You meet people; you c...


In [22]:
conversations = []
for _ in range(100):
    conversations.append({random.choice([row for row in df['_id']]): random.choice([row for row in quote['quote']])})

In [23]:
conversation_random = random.choice([row for row in df_chat['_id']])

In [24]:
for dic in conversations:
    for key,values in dic.items():
        res = requests.get(f'http://localhost:3000/chat/{conversation_random}/addmessage?user_id={key}&text={values}')

In [25]:
print(res)

<Response [500]>


In [26]:
res.json()

{'message': "Error. chat doesn't exist", 'status': 'api-error'}

# Get messages list from a chat

In [27]:
conversation_random = random.choice([row for row in df_chat['_id']])

In [28]:
res = requests.get(f'http://localhost:3000/chat/{conversation_random}/list')

In [29]:
print(res)

<Response [500]>


In [30]:
res.json()

{'message': "Error. chat doesn't exist", 'status': 'api-error'}

# Get sentiment analysis

NLP & Text Sentiment Analysis:
​
- [https://www.nltk.org/]
- [https://towardsdatascience.com/basic-binary-sentiment-analysis-using-nltk-c94ba17ae386]
- [https://www.digitalocean.com/community/tutorials/how-to-perform-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk]

In [31]:
conversation_random = random.choice([row for row in df_chat['_id']])

In [32]:
res = requests.get(f'http://localhost:3000/chat/{conversation_random}/sentiment')

In [33]:
res.json()

{'message': "Error. chat doesn't exist", 'status': 'api-error'}

# Get reccomender analysis

In [34]:
user_random = random.choice([row for row in df['_id']])

In [35]:
res = requests.get(f'http://localhost:3000/user/{user_random}/recommend')

In [36]:
print(res)

<Response [500]>


In [37]:
res.json()

{'message': 'The user is not present in any chat', 'status': 'api-error'}